In [1]:
# Set Variables
SW_LAT = -38.88917
SW_LON = 140.84444
NE_LAT = -33.70833
NE_LON = 150.0

PARAMETER_NAME = "Pat1_C_B_1_DailyMean"
PROPERTY_NAME = "Water Temperature"
TIME_PERIOD = "2010/2024"


In [2]:
import requests
from xml.etree.ElementTree import Element, SubElement, tostring

# Define the namespace map to use in the XML
ns = {
    'sos': "http://www.opengis.net/sos/2.0",
    'fes': "http://www.opengis.net/fes/2.0",
    'gml': "http://www.opengis.net/gml/3.2",
    'xsi': "http://www.w3.org/2001/XMLSchema-instance",
    'sams': "http://www.opengis.net/samplingSpatial/2.0"
}

# Define the endpoint URL for the SOS service
url = "http://www.bom.gov.au/waterdata/services"

# Create XML elements for the request
envelope = Element('sos:GetFeatureOfInterest', {
    'service': 'SOS', 'version': '2.0', 'xmlns:sos': ns['sos'], 'xmlns:fes': ns['fes'], 'xmlns:gml': ns['gml'], 'xmlns:xsi': ns['xsi'], 'xmlns:sams': ns['sams']
})

spatial_filter = SubElement(envelope, 'sos:spatialFilter')
bbox = SubElement(spatial_filter, 'fes:BBOX')
value_ref = SubElement(bbox, 'fes:ValueReference')
value_ref.text = 'om:featureOfInterest/*/sams:shape'

envelope_ = SubElement(bbox, 'gml:Envelope', {
    'srsName': 'urn:ogc:def:crs:EPSG::4326'
})
lower_corner = SubElement(envelope_, 'gml:lowerCorner')
upper_corner = SubElement(envelope_, 'gml:upperCorner')

# Insert the actual coordinates for the Murray River bounding box
lower_corner.text = f"{SW_LAT} {SW_LON}"  # Southernmost, westernmost
upper_corner.text = f"{NE_LAT} {NE_LON}"  # Northernmost, easternmost

# Add observed property element
observed_property = SubElement(envelope, 'sos:observedProperty')
observed_property.text = f"http://bom.gov.au/waterdata/services/parameters/{PROPERTY_NAME}"  # Example observed property


# Convert the ElementTree to a string
request_xml = tostring(envelope, 'utf-8')

# Define headers for the POST request
headers = {
    'Content-Type': 'application/xml'
}

# Send the POST request
FeatureOfInterestResponse = requests.post(url, data=request_xml, headers=headers)

# Print the response from the server


In [3]:
import requests
import xml.etree.ElementTree as ET
# Sample XML response from GetFeatureOfInterest
xml_response = FeatureOfInterestResponse.text

# Parse the XML string
root = ET.fromstring(xml_response)

# Extract station IDs
station_ids = [elem.text.split('/')[-1] for elem in root.findall('.//gml:identifier', namespaces={'gml': 'http://www.opengis.net/gml/3.2'})]

print(station_ids) 

#get stations within region


['A4261084', 'A4261264', 'A4261224', 'A4260535', 'A4261091', 'A4261107', 'A4260592', 'A4261085', 'A4261270', 'A4261255', 'A4261272', 'A4260600', 'A4260700', 'A4261225', 'A4261108', 'A4261540', 'A4261538', 'A4261081', 'A4260698', 'A4261088', 'A4261086', 'A4261087', 'A4261095', 'A4261096', 'A4261798', 'A4261799', 'A4260697', 'A4260576', 'A4261063', 'A4261268', 'A4261265', 'A4261055', 'A4260644', 'A4261106', 'A4260645', 'A4261053', 'A4260699', 'A4260595', 'A4260580', 'A4261083', 'A4261263', 'A4261791', 'A4261168', 'A4260537', 'A4261066', 'A4261789', 'A4260643', 'A4260593', 'A4260517', 'A4260512', 'A4260510', 'A4260663', 'A4260573', 'A4260704', 'A4260705', 'A4261025', 'A4261022', 'A4260652', 'A4261023', 'A4260703', 'A4260642', 'A4261024', 'A4261262', 'A4261170', 'A4260579', 'A4261536', 'A4261539']


In [4]:
import requests
from xml.etree import ElementTree as ET
import pandas as pd

# Define the SOS2 endpoint URL
url = "http://www.bom.gov.au/waterdata/services"

data = []  # List to store data for DataFrame


# Define the parameters for the GetObservation request
params = {
    'service': 'SOS',
    'version': '2.0.0',
    'request': 'GetObservation',
    'procedure': f"http://bom.gov.au/waterdata/services/tstypes/{PARAMETER_NAME}",
    'observedProperty': f"http://bom.gov.au/waterdata/services/parameters/{PROPERTY_NAME}",
    'temporalFilter': f"om:phenomenonTime,{TIME_PERIOD}"
}

# Assuming station_ids is defined earlier in the script
for station_id in station_ids:
    print(f"Querying data for station: {station_id}")
    
    # Add the station ID to the parameters
    params['featureOfInterest'] = f"http://bom.gov.au/waterdata/services/stations/{station_id}"
    
    # Make the GetObservation request to the SOS2 endpoint
    response = requests.get(url, params=params, timeout=100)  # Added timeout
    
    if response.status_code == 200:
        # Parse the XML response
        root = ET.fromstring(response.content)
        
        # Find all the MeasurementTVP elements which contain the time-value pairs
        tvp_elements = root.findall('.//{http://www.opengis.net/waterml/2.0}MeasurementTVP')
        
        if not tvp_elements:
            print("No data found for this station.")
        else:
            # Print out the time and value for each MeasurementTVP
            for tvp in tvp_elements:
                time = tvp.find('{http://www.opengis.net/waterml/2.0}time').text
                value = tvp.find('{http://www.opengis.net/waterml/2.0}value').text
                #print(f"  {time}: {value}")
                data.append({'Station': station_id, 'Time': time, 'Value': value})
    else:
        print(f"Failed to retrieve data for station {station_id}. HTTP status code: {response.status_code}")
        print("Response Headers:", response.headers)
        print("Response Body:", response.text)


df = pd.DataFrame(data)

pivot_df = df.pivot_table(index='Time', values='Value', columns='Station', aggfunc='first')

# Show pivoted DataFrame
print(pivot_df)
# Show DataFrame

Querying data for station: A4261084
No data found for this station.
Querying data for station: A4261264
Querying data for station: A4261224
Querying data for station: A4260535
No data found for this station.
Querying data for station: A4261091
Querying data for station: A4261107
Querying data for station: A4260592
No data found for this station.
Querying data for station: A4261085
No data found for this station.
Querying data for station: A4261270
Querying data for station: A4261255
Querying data for station: A4261272
Querying data for station: A4260600
Querying data for station: A4260700
No data found for this station.
Querying data for station: A4261225
Querying data for station: A4261108
Querying data for station: A4261540
Querying data for station: A4261538
Querying data for station: A4261081
No data found for this station.
Querying data for station: A4260698
No data found for this station.
Querying data for station: A4261088
No data found for this station.
Querying data for statio

In [5]:
pivot_df.to_csv('DailyMeanTempData.csv', index=True)